In [ ]:
문제 풀기: ~11:00

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv('./test/mobiles.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   screen_size       430 non-null    object 
 1   ROM               430 non-null    int64  
 2   RAM               430 non-null    int64  
 3   num_rear_camera   430 non-null    int64  
 4   num_front_camera  430 non-null    int64  
 5   battery_capacity  430 non-null    int64  
 6   ratings           430 non-null    float64
 7   num_of_ratings    430 non-null    int64  
 8   sales_price       430 non-null    int64  
 9   discount_percent  430 non-null    float64
 10  sales             430 non-null    float64
dtypes: float64(3), int64(7), object(1)
memory usage: 37.1+ KB


# Q1

In [3]:
xbar=df.sales.mean()
std1=df.sales.std()

ub=xbar + (2 * std1)

q1=df[df.sales > ub].reset_index(drop=True)

len(df), len(q1)

(430, 16)

In [4]:
q1['카메라개수']=q1['num_rear_camera'] + q1['num_front_camera']
q1['성능지표']= (q1['ROM'] / 32) + (q1['RAM'] / 2) + q1['카메라개수'] + (q1['battery_capacity'] / 1000) 

In [5]:
round(q1['성능지표'].mean(),2)

11.01

# Q2

In [6]:
q2=df[df.num_rear_camera > 1].reset_index(drop=True)

In [7]:
list2=['battery_capacity', 'ratings', 'num_of_ratings', 'sales_price',
       'discount_percent', 'sales']

In [11]:
q2[list2].corr()['sales'].drop('sales').abs().round(2).max()

0.95

# Q3

In [12]:
len(df.columns)

11

In [13]:
df.dtypes

screen_size          object
ROM                   int64
RAM                   int64
num_rear_camera       int64
num_front_camera      int64
battery_capacity      int64
ratings             float64
num_of_ratings        int64
sales_price           int64
discount_percent    float64
sales               float64
dtype: object

In [14]:
df.screen_size.unique()

array(['Very Small', 'Small', 'Medium', 'Large', 'Very Large'],
      dtype=object)

- pd.get_dummies(데이터셋, columns=[원핫으로 바꿀 변수명 나열], drop_first=False) # 구버전
- pd.get_dummies(데이터셋, columns=[원핫으로 바꿀 변수명 나열], drop_first=False, dtype='int') # 신버전

In [17]:
q3=pd.get_dummies(df, columns=['screen_size'], drop_first=False, dtype='int')

In [18]:
q3.head(2)

,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
0,64,2,1,1,1800,4.5,38645,32999,0.17,127.52,0,0,0,0,1
1,64,4,2,1,2815,4.5,244,57149,0.04,1.39,0,0,1,0,0


In [19]:
len(q3.columns)

15

In [20]:
train, test=train_test_split(q3, test_size=0.2, random_state=123)

In [22]:
minmax=MinMaxScaler().fit(train)
train_m=pd.DataFrame(minmax.transform(train), columns=q3.columns)
test_m=pd.DataFrame(minmax.transform(test), columns=q3.columns)

In [ ]:
knn=KNeighborsRegressor(n_neighbors=3)
knn.fit(train_m.drop(columns='sales'), train_m['sales'])
pred=knn.predict(test_m.drop(columns='sales'))
rmse=np.sqrt(mean_squared_error(test_m['sales'], pred))

In [23]:
rmse_col=[]

for i in [3,5,7,9,11]:
    knn=KNeighborsRegressor(n_neighbors=i)
    knn.fit(train_m.drop(columns='sales'), train_m['sales'])
    pred=knn.predict(test_m.drop(columns='sales'))
    rmse=np.sqrt(mean_squared_error(test_m['sales'], pred))
    rmse_col.append([i, rmse])

In [26]:
rmse_col = pd.DataFrame(rmse_col, columns=['k','rmse'])
rmse_col

,k,rmse
0,3,0.081867
1,5,0.098791
2,7,0.107670
3,9,0.112321
4,11,0.113690


In [28]:
best_k=rmse_col.loc[rmse_col.rmse.idxmin(),'k']
best_k

3

In [ ]:
knn_k=KNeighborsRegressor(n_neighbors=3)
knn_k.fit(train_m.drop(columns='sales'), train_m['sales'])
pred=knn_k.predict(test_m.drop(columns='sales'))
rmse=np.sqrt(mean_squared_error(test_m['sales'], pred))

Q3. [추가 지시사항] 다음은 저번달에 신규 출시된 경쟁사의 스마트폰 정보이다. 해당 스마트폰의 판매지수는 얼마로 예상되는가?
- 단계 1) 주어진 데이터는 기존 원핫인코딩 규칙을 기반으로 더미변수를 생성하시오.
- 단계 2) 기존 정규화 규칙을 기반으로 주어진 데이터를 정규화 하시오.
- 단계 3) 기존에 정제한 학습 데이터 세트와 이웃 개수는 직전에 최적이라고 판단한 k값을 사용한 k-NN 모델을 준비하시오.
- 단계 4) 준비된 k-NN 모델에 "단계 2"에서 산출한 데이터 세트를 입력하고 그 결과를 확인하시오.
- 단계 5) "단계 4"의 결과물을 기존 정규화 규칙을 기반으로 역변환 하시오.

   - 기존에 정제한 학습 데이터 세트와 이웃 개수는 직전에 최적이라고 판단한 k값을 사용하여 학습한 k-NN 모델을 사용하시오.
    - k-NN 모델로 산출되는 값은 기존에 생성한 정규화 규칙을 기반으로 역변환 하시오.
    - 정답은 반올림하여 소수 첫째 자리까지 출력하시오. (정답 예시: 0.1)

- ROM: 256
- RAM: 6
- num_rear_camera: 4
- num_front_camera: 1
- battery_capacity: 4000
- ratings: 4.3
- num_of_ratings: 25000
- sales_price: 85000
- discount_percent: 0.05
- screen_size: "Large"

In [34]:
knn_k=KNeighborsRegressor(n_neighbors=best_k)
knn_k.fit(train_m.drop(columns='sales'), train_m['sales'])
#pred=knn_k.predict(test_m.drop(columns='sales'))

KNeighborsRegressor(n_neighbors=3)

In [31]:
q3_2=train.head(1).copy()
q3_2

,ROM,RAM,num_rear_camera,num_front_camera,battery_capacity,ratings,num_of_ratings,sales_price,discount_percent,sales,screen_size_Large,screen_size_Medium,screen_size_Small,screen_size_Very Large,screen_size_Very Small
410,32,3,2,1,4000,4.3,1870,10449,0.05,1.95,0,0,1,0,0


In [32]:
q3_2['ROM'] = 256
q3_2['RAM'] = 6
q3_2['num_rear_camera']=4
q3_2['num_front_camera'] = 1
q3_2['battery_capacity'] = 4000
q3_2['ratings']=4.3
q3_2['num_of_ratings'] = 25000
q3_2['sales_price'] = 85000
q3_2['discount_percent'] = 0.05
q3_2['screen_size_Large'] = 1
q3_2['screen_size_Medium'] = 0
q3_2['screen_size_Small'] = 0
q3_2['screen_size_Very Large'] = 0
q3_2['screen_size_Very Small'] = 0

In [33]:
test_vm=pd.DataFrame(minmax.transform(q3_2), columns=q3.columns)

In [35]:
test_vm['sales']=knn_k.predict(test_vm.drop(columns='sales'))

In [37]:
pd.DataFrame(minmax.inverse_transform(test_vm), columns=q3.columns)['sales']

0    0.653333
Name: sales, dtype: float64